## Image Alteration Detection

#### Import libraries

In [8]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

#### Defining required parameters

In [9]:
# Parameters
IMG_SIZE = (128, 128)  # Smaller input for faster training
BATCH_SIZE = 16
EPOCHS = 10  # increase this for better results

# Paths
train_dir = 'dataset/train'
test_dir = 'dataset/test_mixed'

#### Generating data

In [10]:
# Data Generator
train_gen = ImageDataGenerator(preprocessing_function=preprocess_input,
                               validation_split=0.2)

train_data = train_gen.flow_from_directory(train_dir,
                                           target_size=IMG_SIZE,
                                           batch_size=BATCH_SIZE,
                                           class_mode='binary',
                                           subset='training')

val_data = train_gen.flow_from_directory(train_dir,
                                         target_size=IMG_SIZE,
                                         batch_size=BATCH_SIZE,
                                         class_mode='binary',
                                         subset='validation')

Found 80000 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.


#### Creating model

In [11]:
# Build Model with ResNet50

# Load weights from local path

# Assuming the file is in the same directory as your script
local_weights_file = os.path.abspath("resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5")

base_model = ResNet50(weights=local_weights_file, include_top=False, input_shape=(128, 128, 3))
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(64, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=output)

# Freeze base model
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])

#### Training model

In [12]:
# Train
history = model.fit(train_data, validation_data=val_data, epochs=EPOCHS)

Epoch 1/10
  82/5000 ━━━━━━━━━━━━━━━━━━━━ 51:57 634ms/step - accuracy: 0.5727 - loss: 0.7301

KeyboardInterrupt: 

#### Predicting model

In [ ]:
# Predict on mixed test folder
def predict_mixed_folder(test_dir):
    predictions = []
    files = os.listdir(test_dir)
    for fname in files:
        img_path = os.path.join(test_dir, fname)
        img = load_img(img_path, target_size=IMG_SIZE)
        img_array = img_to_array(img)
        img_array = preprocess_input(img_array)
        img_array = np.expand_dims(img_array, axis=0)

        prob = model.predict(img_array)[0][0]
        label = 'Fake' if prob > 0.5 else 'Real'
        predictions.append((fname, label, prob))

    return predictions

#### Showing the result

In [7]:
# Run and show results
results = predict_mixed_folder(test_dir)
for fname, label, prob in results:
    print(f"{fname}: {label} ({prob:.2f})")

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 